In [132]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm

# **Function for Max Drawdown Calculation (in % and in days)**

In [141]:
def calculate_max_drawdown_and_duration(monthly_returns):
    """
    Calculate the maximum drawdown (in percentage) and the longest drawdown duration (in days).

    Parameters:
    - monthly_returns (list or array-like): A list of monthly portfolio returns.

    Returns:
    - max_drawdown (float): The maximum drawdown as a percentage.
    - longest_drawdown_days (float): The longest drawdown duration in days.
    """
    # Convert monthly returns to a pandas Series
    returns_series = pd.Series(monthly_returns)

    # Calculate the cumulative returns
    cumulative_returns = (1 + returns_series).cumprod()

    # Calculate the rolling maximum of the cumulative returns
    rolling_max = cumulative_returns.cummax()

    # Calculate drawdowns
    drawdowns = (cumulative_returns / rolling_max) - 1

    # Maximum drawdown
    max_drawdown = drawdowns.min()

    # Identify the longest drawdown duration
    drawdown_periods = drawdowns[drawdowns < 0]  # Filter periods with drawdowns
    longest_drawdown_months = 0
    current_duration = 0

    for i in range(len(drawdowns)):
        if drawdowns[i] < 0:
            current_duration += 1
            longest_drawdown_months = max(longest_drawdown_months, current_duration)
        else:
            current_duration = 0

    # Convert months to days (average of 30.44 days per month)
    longest_drawdown_days = longest_drawdown_months * 30.44

    return max_drawdown, longest_drawdown_days


In [142]:
# Load data
SPY_DATA = pd.read_csv("backtest_long_only.csv")
EWP_DATA = pd.read_csv("backtest_EWP.csv")
LONG_ONLY_DATA = pd.read_csv("backtest_long_only.csv")
LONG_SHORT_DATA = pd.read_csv("backtest_long_short.csv")

# **SPY Benchmark Portfolio**

In [143]:
BM_daily_rets = LONG_ONLY_DATA["SPY_Returns"]
BM_daily_vol = np.sqrt(np.var(BM_daily_rets))

BM_annual_vol = BM_daily_vol*np.sqrt(12)

BM_Cum_Return = ((BM_daily_rets+np.ones(len(BM_daily_rets))).prod()-1)
BM_Annualized_Return = ((BM_daily_rets+np.ones(len(BM_daily_rets))).prod()**(12/len(BM_daily_rets))-1)

rf = 0 # risk-free rate assumed to be zero

BM_Sharpe_Ratio = (BM_Annualized_Return - rf) / BM_annual_vol

BM_max_drawdown, BM_longest_drawdown_days = calculate_max_drawdown_and_duration(BM_daily_rets)

print("SPY Annualized Volatility: ", round(BM_annual_vol*100,3), "%")
print("SPY Cumulative Returns: ", round(BM_Cum_Return*100,3), "%")
print("SPY Annualized Return: ", round(BM_Annualized_Return*100,3), "%")
print("SPY Sharpe Ratio: ", round(BM_Sharpe_Ratio,2))
print(f"SPY Maximum Drawdown: {BM_max_drawdown:.2f}%")
print(f"SPY Longest Drawdown Duration: {BM_longest_drawdown_days:.0f} days")
print("SPY Beta: ", 1)
print("SPY Alpha: ", 0.00, "%")

SPY Annualized Volatility:  15.422 %
SPY Cumulative Returns:  270.533 %
SPY Annualized Return:  5.491 %
SPY Sharpe Ratio:  0.36
SPY Maximum Drawdown: -0.52%
SPY Longest Drawdown Duration: 2435 days
SPY Beta:  1
SPY Alpha:  0.0 %


# **Equal Weight Portfolio**

In [144]:
EWP_daily_rets = EWP_DATA["EQ_Returns"]
EWP_daily_vol = np.sqrt(np.var(EWP_daily_rets))

EWP_annual_vol = EWP_daily_vol*np.sqrt(12)

EWP_Cum_Return = ((EWP_daily_rets+np.ones(len(EWP_daily_rets))).prod()-1)
EWP_Annualized_Return = ((EWP_daily_rets+np.ones(len(EWP_daily_rets))).prod()**(12/len(EWP_daily_rets))-1)

rf = 0 # risk-free rate assumed to be zero

EWP_Sharpe_Ratio = (EWP_Annualized_Return - rf) / EWP_annual_vol

EWP_max_drawdown, EWP_longest_drawdown_days = calculate_max_drawdown_and_duration(EWP_daily_rets)

data_EWP = {'SPY': BM_daily_rets,
            'EWP': EWP_daily_rets}
df_EWP = pd.DataFrame(data_EWP)
X_EWP = sm.add_constant(df_EWP[['SPY']])
Y_EWP = df_EWP['EWP']
model_EWP = sm.OLS(Y_EWP, X_EWP).fit()
beta_EWP = model_EWP.params['SPY']
alpha_EWP = model_EWP.params['const']


print("EWP Annualized Volatility: ", round(EWP_annual_vol*100,3), "%")
print("EWP Cumulative Returns: ", round(EWP_Cum_Return*100,3), "%")
print("EWP Annualized Return: ", round(EWP_Annualized_Return*100,3), "%")
print("EWP Sharpe Ratio: ", round(EWP_Sharpe_Ratio,2))
print(f"EWP Maximum Drawdown: {EWP_max_drawdown:.2f}%")
print(f"EWP Longest Drawdown Duration: {EWP_longest_drawdown_days:.0f} days")
print("EWP Beta: ", round(beta_EWP,2))
print("EWP Alpha: ", round(alpha_EWP*100,2), "%")

EWP Annualized Volatility:  14.511 %
EWP Cumulative Returns:  179.675 %
EWP Annualized Return:  4.287 %
EWP Sharpe Ratio:  0.3
EWP Maximum Drawdown: -0.45%
EWP Longest Drawdown Duration: 1522 days
EWP Beta:  0.88
EWP Alpha:  -0.04 %


# **Long Only Portfolio**

In [145]:
LO_daily_rets = LONG_ONLY_DATA["Strategy_Returns"]
LO_daily_vol = np.sqrt(np.var(LO_daily_rets))

LO_annual_vol = LO_daily_vol*np.sqrt(12)

LO_Cum_Return = ((LO_daily_rets+np.ones(len(LO_daily_rets))).prod()-1)
LO_Annualized_Return = ((LO_daily_rets+np.ones(len(LO_daily_rets))).prod()**(12/len(LO_daily_rets))-1)

rf = 0 # risk-free rate assumed to be zero

LO_Sharpe_Ratio = (LO_Annualized_Return - rf) / LO_annual_vol

LO_max_drawdown, LO_longest_drawdown_days = calculate_max_drawdown_and_duration(LO_daily_rets)

data_LO = {'SPY': BM_daily_rets,
            'LO': LO_daily_rets}
df_LO = pd.DataFrame(data_LO)
X_LO = sm.add_constant(df_LO[['SPY']])
Y_LO = df_LO['LO']
model_LO = sm.OLS(Y_LO, X_LO).fit()
beta_LO = model_LO.params['SPY']
alpha_LO = model_LO.params['const']

print("Long-Only Annualized Volatility: ", round(LO_annual_vol*100,3), "%")
print("Long-Only Cumulative Returns: ", round(LO_Cum_Return*100,3), "%")
print("Long-Only Annualized Return: ", round(LO_Annualized_Return*100,3), "%")
print("Long-Only Sharpe Ratio: ", round(LO_Sharpe_Ratio,2))
print(f"Long-Only Maximum Drawdown: {LO_max_drawdown:.2f}%")
print(f"Long-Only Longest Drawdown Duration: {LO_longest_drawdown_days:.0f} days")
print("Long-Only Beta: ", round(beta_LO,2))
print("Long-Only Alpha: ", round(alpha_LO*100,2), "%")

Long-Only Annualized Volatility:  19.374 %
Long-Only Cumulative Returns:  437.964 %
Long-Only Annualized Return:  7.109 %
Long-Only Sharpe Ratio:  0.37
Long-Only Maximum Drawdown: -0.47%
Long-Only Longest Drawdown Duration: 2709 days
Long-Only Beta:  0.64
Long-Only Alpha:  0.38 %


# **Long-Short Portfolio**

In [146]:
from re import A
LS_daily_rets = LONG_SHORT_DATA["Strategy_Returns"]
LS_daily_vol = np.sqrt(np.var(LS_daily_rets))

LS_annual_vol = LS_daily_vol*np.sqrt(12)

LS_Cum_Return = ((LS_daily_rets+np.ones(len(LS_daily_rets))).prod()-1)
LS_Annualized_Return = ((LS_daily_rets+np.ones(len(LS_daily_rets))).prod()**(12/len(LS_daily_rets))-1)

rf = 0 # risk-free rate assumed to be zero

LS_Sharpe_Ratio = (LS_Annualized_Return - rf) / LS_annual_vol

LS_max_drawdown, LS_longest_drawdown_days = calculate_max_drawdown_and_duration(LS_daily_rets)

data_LS = {'SPY': BM_daily_rets,
            'LS': LS_daily_rets}
df_LS = pd.DataFrame(data_LS)
X_LS = sm.add_constant(df_LS[['SPY']])
Y_LS = df_LS['LS']
model_LS = sm.OLS(Y_LS, X_LS).fit()
beta_LS = model_LS.params['SPY']
alpha_LS = model_LS.params['const']

print("Long-Short Annualized Volatility: ", round(LS_annual_vol*100,3), "%")
print("Long-Short Cumulative Returns: ", round(LS_Cum_Return*100,3), "%")
print("Long-Short Annualized Return: ", round(LS_Annualized_Return*100,3), "%")
print("Long-Short Sharpe Ratio: ", round(LS_Sharpe_Ratio,2))
print(f"Long-Short Maximum Drawdown: {LS_max_drawdown:.2f}%")
print(f"Long-Short Longest Drawdown Duration: {LS_longest_drawdown_days:.0f} days")
print("Long-Short Beta: ", round(beta_LS,2))
print("Long-Short Alpha: ", round(alpha_LS*100,2), "%")

Long-Short Annualized Volatility:  17.024 %
Long-Short Cumulative Returns:  520.939 %
Long-Short Annualized Return:  7.738 %
Long-Short Sharpe Ratio:  0.45
Long-Short Maximum Drawdown: -0.56%
Long-Short Longest Drawdown Duration: 4475 days
Long-Short Beta:  0.17
Long-Short Alpha:  0.65 %


# **Now Print Everything in a Table**

In [147]:
Table_data = {
    'Metric': ['Cumulative Return', 'Annualized Return', 'Annualized Volatility',
               'Sharpe Ratio', 'Maximum Drawdown', 'Longest Drawdown Duration (days)',
               'Beta', 'Alpha'],

    'SPY': [BM_Cum_Return, BM_Annualized_Return, BM_annual_vol, BM_Sharpe_Ratio, BM_max_drawdown, BM_longest_drawdown_days, 1.0, 0.0],
    'Equal Weight Portfolio': [EWP_Cum_Return, EWP_Annualized_Return, EWP_annual_vol, EWP_Sharpe_Ratio, EWP_max_drawdown, EWP_longest_drawdown_days, beta_EWP, alpha_EWP],
    'Long Only': [LO_Cum_Return, LO_Annualized_Return, LO_annual_vol, LO_Sharpe_Ratio, LO_max_drawdown, LO_longest_drawdown_days, beta_LO, alpha_LO],
    'Long-Short': [LS_Cum_Return, LS_Annualized_Return, LS_annual_vol, LS_Sharpe_Ratio, LS_max_drawdown, LS_longest_drawdown_days, beta_LS, alpha_LS]
}

Table_df = pd.DataFrame(Table_data)
Table_df.set_index('Metric', inplace=True)
print(Table_df)

                                          SPY  Equal Weight Portfolio  \
Metric                                                                  
Cumulative Return                    2.705327                1.796755   
Annualized Return                    0.054915                0.042871   
Annualized Volatility                0.154223                0.145111   
Sharpe Ratio                         0.356074                0.295440   
Maximum Drawdown                    -0.521953               -0.447653   
Longest Drawdown Duration (days)  2435.200000             1522.000000   
Beta                                 1.000000                0.878195   
Alpha                                0.000000               -0.000408   

                                    Long Only   Long-Short  
Metric                                                      
Cumulative Return                    4.379639     5.209394  
Annualized Return                    0.071092     0.077381  
Annualized Volatility    

In [150]:
import pandas as pd

# Your table data
Table_data = {
    'Metric': ['Cumulative Return', 'Annualized Return', 'Annualized Volatility',
               'Sharpe Ratio', 'Maximum Drawdown', 'Longest Drawdown Duration (days)',
               'Beta', 'Alpha'],

    'SPY': [BM_Cum_Return, BM_Annualized_Return, BM_annual_vol, BM_Sharpe_Ratio, BM_max_drawdown, BM_longest_drawdown_days, 1.0, 0.0],
    'Equal Weight Portfolio': [EWP_Cum_Return, EWP_Annualized_Return, EWP_annual_vol, EWP_Sharpe_Ratio, EWP_max_drawdown, EWP_longest_drawdown_days, beta_EWP, alpha_EWP],
    'Long Only': [LO_Cum_Return, LO_Annualized_Return, LO_annual_vol, LO_Sharpe_Ratio, LO_max_drawdown, LO_longest_drawdown_days, beta_LO, alpha_LO],
    'Long-Short': [LS_Cum_Return, LS_Annualized_Return, LS_annual_vol, LS_Sharpe_Ratio, LS_max_drawdown, LS_longest_drawdown_days, beta_LS, alpha_LS]
}

# Create DataFrame
Table_df = pd.DataFrame(Table_data)

# Set 'Metric' as the index
Table_df.set_index('Metric', inplace=True)

# Apply styling for visual appeal
styled_df = Table_df.style.set_table_styles(
    [{'selector': 'thead th', 'props': [('font-weight', 'bold'), ('text-align', 'center'),
                                        ('background-color', 'black'), ('color', 'white')]},  # Black background, white font
     {'selector': 'tbody td', 'props': [('text-align', 'center')]},  # Center all cell values
     {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%')]},  # Border collapse and full-width table
     {'selector': 'tr', 'props': [('border-bottom', '1px solid #ddd')]},  # Light borders for rows
     {'selector': 'td', 'props': [('padding', '10px')]},  # Padding for better spacing
    ]
)

# Display the styled table
styled_df


,SPY,Equal Weight Portfolio,Long Only,Long-Short
Metric,,,,
Cumulative Return,2.705327,1.796755,4.379639,5.209394
Annualized Return,0.054915,0.042871,0.071092,0.077381
Annualized Volatility,0.154223,0.145111,0.193740,0.170237
Sharpe Ratio,0.356074,0.295440,0.366944,0.454548
Maximum Drawdown,-0.521953,-0.447653,-0.465239,-0.560229
Longest Drawdown Duration (days),2435.200000,1522.000000,2709.160000,4474.680000
Beta,1.000000,0.878195,0.638757,0.174943
Alpha,0.000000,-0.000408,0.003809,0.006478
